In [1]:
import ipyparallel as ipp
cluster = ipp.Cluster(n=1, engines="mpi")
rc = cluster.start_and_connect_sync(activate=True)

Starting 1 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/1 [00:00<?, ?engine/s]

In [2]:
def apply_imports(cluster):
    with cluster[:].sync_imports():
        import dolfinx
        from mpi4py import MPI
        import time
        import ufl
        import numpy

In [3]:
def solve_poisson(N):
    MPI.COMM_WORLD.Barrier()
    start = time.perf_counter()
    mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, N, N)
    V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)
    a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
    x = ufl.SpatialCoordinate(mesh)
    f = ufl.cos(2*ufl.pi*x[0])
    L = ufl.inner(f, v) * ufl.dx
    mesh.topology.create_connectivity(mesh.topology.dim-1, mesh.topology.dim)
    bndry_facets = dolfinx.mesh.exterior_facet_indices(mesh.topology)
    bndry_dofs = dolfinx.fem.locate_dofs_topological(
        V, mesh.topology.dim-1, bndry_facets)
    bc = [dolfinx.fem.dirichletbc(numpy.float64(0), bndry_dofs, V)]
    petsc_options = {"ksp_type": "cg",
                     "pc_type": "hypre",
                     "pc_hypre_type": "boomeramg",
                     "ksp_rtol": 1.0e-8,
                     "pc_hypre_boomeramg_strong_threshold": 0.7,
                     "pc_hypre_boomeramg_agg_nl": 4,
                     "pc_hypre_boomeramg_agg_num_paths": 2}
    end = time.perf_counter()

    problem = dolfinx.fem.petsc.LinearProblem(
        a, L, bc, petsc_options=petsc_options)
    uh = problem.solve()

    local_cntr = dolfinx.fem.assemble_scalar(dolfinx.fem.form(uh*uh*ufl.dx))
    global_cntr = mesh.comm.reduce(local_cntr, op=MPI.SUM, root=0)
    it = problem.solver.getIterationNumber()
    if mesh.comm.rank == 0:
        print(
            f"Num processes: {mesh.comm.size} Num dofs: {V.dofmap.index_map.size_global} Num iterations: {it} int u^2: {global_cntr}")
    return end-start

In [4]:
apply_imports(rc)

importing dolfinx on engine(s)
importing MPI from mpi4py on engine(s)
importing time on engine(s)
importing ufl on engine(s)
importing numpy on engine(s)


In [5]:
result_serial = rc[:].apply_async(solve_poisson, 300)
result_serial.wait()
print(result_serial.stdout, result_serial.get())
await cluster.stop_engines()

['Num processes: 1 Num dofs: 90601 Num iterations: 9 int u^2: 0.00021415468086530374\n'] [0.2504483610000534]
Stopping engine(s): 1681721444


INFO:ipyparallel.cluster.cluster.1681721443-hz28:Stopping engine(s): 1681721444


Failed to remove /home/dokken/.ipython/profile_default/log/ipengine-1681721443-hz28-1681721444-46787.log: [Errno 2] No such file or directory: '/home/dokken/.ipython/profile_default/log/ipengine-1681721443-hz28-1681721444-46787.log'


ERROR:ipyparallel.cluster.cluster.1681721443-hz28:Failed to remove /home/dokken/.ipython/profile_default/log/ipengine-1681721443-hz28-1681721444-46787.log: [Errno 2] No such file or directory: '/home/dokken/.ipython/profile_default/log/ipengine-1681721443-hz28-1681721444-46787.log'


mpiexec error output:
=   BAD TERMINATION OF ONE OF YOUR APPLICATION PROCESSES
=   PID 46866 RUNNING AT dokken-XPS-9320
=   EXIT CODE: 139
=   CLEANING UP REMAINING PROCESSES
=   YOU CAN IGNORE THE BELOW CLEANUP MESSAGES
YOUR APPLICATION TERMINATED WITH THE EXIT STRING: Segmentation fault (signal 11)



=   BAD TERMINATION OF ONE OF YOUR APPLICATION PROCESSES
=   PID 46866 RUNNING AT dokken-XPS-9320
=   EXIT CODE: 139
=   CLEANING UP REMAINING PROCESSES
=   YOU CAN IGNORE THE BELOW CLEANUP MESSAGES
YOUR APPLICATION TERMINATED WITH THE EXIT STRING: Segmentation fault (signal 11)



engine set stopped 1681721444: {'exit_code': 139, 'pid': 46861, 'identifier': 'ipengine-1681721443-hz28-1681721444-46787'}


In [ ]:
import numpy
mpi_processes = [1, 2, 4, 8]
runtimes = numpy.zeros(len(mpi_processes))
for i, processes in enumerate(mpi_processes):
    cluster = ipp.Cluster(n=processes, engines="mpi")
    rc = cluster.start_and_connect_sync(activate=True)
    apply_imports(rc)
    process_runtimes = rc[:].apply_async(solve_poisson, 50)
    process_runtimes.wait()
    runtimes[i] = numpy.sum(process_runtimes.get())/len(process_runtimes.get())
    print(runtimes[i], process_runtimes.stdout)
    await cluster.stop_engines()

Starting 1 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


INFO:ipyparallel.cluster.cluster.1681721451-uxj8:Starting 1 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/1 [00:00<?, ?engine/s]

importing dolfinx on engine(s)
importing MPI from mpi4py on engine(s)
importing time on engine(s)
importing ufl on engine(s)
importing numpy on engine(s)
0.009200198999678832 ['Num processes: 1 Num dofs: 2601 Num iterations: 7 int u^2: 0.00021357170087765075\n']
Stopping engine(s): 1681721452


INFO:ipyparallel.cluster.cluster.1681721451-uxj8:Stopping engine(s): 1681721452


...

=   BAD TERMINATION OF ONE OF YOUR APPLICATION PROCESSES
=   PID 47053 RUNNING AT dokken-XPS-9320
=   EXIT CODE: 9
=   CLEANING UP REMAINING PROCESSES
=   YOU CAN IGNORE THE BELOW CLEANUP MESSAGES
YOUR APPLICATION TERMINATED WITH THE EXIT STRING: Segmentation fault (signal 11)



engine set stopped 1681721459: {'exit_code': 139, 'pid': 47050, 'identifier': 'ipengine-1681721458-gv9v-1681721459-46787'}


Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


INFO:ipyparallel.cluster.cluster.1681721465-tcti:Starting 4 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/4 [00:00<?, ?engine/s]

In [ ]:
import matplotlib.pyplot as plt
print("Average runtimes")
for num_processes, runtime in zip(mpi_processes, runtimes):
    print(num_processes, runtime)
plt.plot(mpi_processes, runtimes)
plt.xlabel("Number of processes")
plt.ylabel("Runtime")
plt.grid()

In [ ]:
cluster = ipp.Cluster(n=processes, engines="mpi")
rc = cluster.start_and_connect_sync(activate=True)
apply_imports(rc)

In [ ]:
%%px

N = 100
MPI.COMM_WORLD.Barrier()
start = time.perf_counter()
mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, N, N)
V = dolfinx.fem.FunctionSpace(mesh, ("Lagrange", 1))
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
x = ufl.SpatialCoordinate(mesh)
f = ufl.cos(2*ufl.pi*x[0])
L = ufl.inner(f, v) * ufl.dx
mesh.topology.create_connectivity(mesh.topology.dim-1, mesh.topology.dim)
bndry_facets = dolfinx.mesh.exterior_facet_indices(mesh.topology)
bndry_dofs = dolfinx.fem.locate_dofs_topological(
    V, mesh.topology.dim-1, bndry_facets)
bc = [dolfinx.fem.dirichletbc(numpy.float64(0), bndry_dofs, V)]
petsc_options = {"ksp_type": "cg",
                 "pc_type": "hypre",
                 "pc_hypre_type": "boomeramg",
                 "ksp_rtol": 1.0e-8,
                 "pc_hypre_boomeramg_strong_threshold": 0.7,
                 "pc_hypre_boomeramg_agg_nl": 4,
                 "pc_hypre_boomeramg_agg_num_paths": 2}
end = time.perf_counter()

problem = dolfinx.fem.petsc.LinearProblem(
    a, L, bc, petsc_options=petsc_options)
uh = problem.solve()

local_cntr = dolfinx.fem.assemble_scalar(dolfinx.fem.form(uh*uh*ufl.dx))
global_cntr = mesh.comm.reduce(local_cntr, op=MPI.SUM, root=0)
it = problem.solver.getIterationNumber()
if mesh.comm.rank == 0:
    print(
        f"Num processes: {mesh.comm.size} Num dofs: {V.dofmap.index_map.size_global} Num iterations: {it} int u^2: {global_cntr}")
print(end-start)